# Part 2: Array-oriented programming and control flow

_(To be viewed with [jupyterlab-deck](https://jupyterlab-deck.readthedocs.io/).)_

## Single Instruction, Multiple Data (SIMD)

<br>

Modern CPUs have two types of registers for low-level hardware instructions:

  * scalar registers, which hold one numeric value at a time
  * vector registers, which hold more than one numeric value at a time (such as 4 or 1024)

(GPUs have only vector registers, and they're bigger.)

Here's a little simulation in Python:

In [ ]:
class ScalarRegister:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return f"ScalarRegister({self.value})"
    def __add__(self, other):
        assert isinstance(other, ScalarRegister)
        return ScalarRegister(self.value + other.value)

class VectorRegister:
    def __init__(self, value1, value2, value3, value4):
        self.value1, self.value2, self.value3, self.value4 = value1, value2, value3, value4
    def __repr__(self):
        return f"VectorRegister({self.value1}, {self.value2}, {self.value3}, {self.value4})"
    def __add__(self, other):
        assert isinstance(other, VectorRegister)
        return VectorRegister(
            self.value1 + other.value1, self.value2 + other.value2,
            self.value3 + other.value3, self.value4 + other.value4,
        )

In [ ]:
ram_memory = [  1,   2,   3,   4,   5,   6,   7,   8,   9,   10,   11,   12,
              100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200]

<br>

In [ ]:
left = ram_memory[0:12]
right = ram_memory[12:24]

for i in range(0, 12):
    register_A = ScalarRegister(left[i])
    register_B = ScalarRegister(right[i])
    print(f"time step: {register_A + register_B}")

In [ ]:
for i in range(0, 12, 4):
    register_A = VectorRegister(left[i], left[i + 1], left[i + 2], left[i + 3])
    register_B = VectorRegister(right[i], right[i + 1], right[i + 2], right[i + 3])
    print(f"time step: {register_A + register_B}")

<br><br><br>

A vector register that holds 4 numbers at a time gets operated on by an addition instruction that computes 4 additions at once, nominally 4 times faster.

(In practice, the processor may run at a slower rate during vector processing, and other complications...)

The drawback is that these instructions can _only_ perform 4 additions at a time.

<br>

You can't choose whether or not to do the 4th addition based on the result of the 3rd, for instance.

<br>

This is why a compiler might tell you that your code can't be compiled with vectorization, especially if it includes control flow structures like `if` and `for`.

Array operations in libraries like NumPy are also SIMD: **Single (Python) Instruction, applied to Multiple Data**.

<br>

Whereas hardware SIMD can be faster than scalar instructions because a fixed number of operations are performed in the same time step, NumPy is usually faster than pure Python because its operations are compiled.

<br>

Array-oriented programming, "high-level SIMD," has the same limitations as low-level SIMD: an instruction on all elements of an array can't be modified by inserting control flow (`if` and `for`) between operations on elements.

## Illustration: computing special functions

For a moment, let's suppose that SciPy doesn't exist and you want to compute the log of the Gamma function,

$$\log\Gamma(z) \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} \Gamma(z) = \int_0^\infty t^{z - 1} e^{-t} \, dt$$

So you dust off your copy of [_Numerical Recipes_](http://numerical.recipes/). (You have one, right?)

<center>
<img src="../img/numerical-recipes.jpg" width="25%">
</center>

Inside (page 214), you find the algorithm it suggests and translate it from Fortran or Pascal or whatever into Python:

In [ ]:
def log_of_gamma(x):
    tmp = x + 5.5
    tmp -= (x + 0.5) * np.log(tmp)
    series = 1.000000000190015
    for i, coefficient in enumerate([
        76.18009172947146, -86.50532032941677, 24.01409824083091,
        -1.231739572450155, 0.1208650973866179e-2, -0.5395239384953e-5,
    ]):
        series += coefficient / (x + i + 1)
    return -tmp + np.log(2.5066282746310005 * series / x)

<br><br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
xs = np.linspace(0, 10, 10000)[1:]
ys = [log_of_gamma(x) for x in xs]  # this list comprehension is a Python for loop!
plt.plot(xs, ys);

SciPy _does_ exist, so we confirm the calculation by overlaying it.

In [ ]:
import scipy.special
plt.plot(xs, ys); plt.plot(xs, scipy.special.loggamma(xs));

Our function, `log_of_gamma`, works just as well for arrays as it does for scalars, but faster.

In [ ]:
plt.plot(xs, log_of_gamma(xs));  # vectorized across the array, not a Python for loop

The array could replace a scalar because every operation is a [closed-form](https://en.wikipedia.org/wiki/Closed-form_expression) expression.

```python
def log_of_gamma(x):
    tmp = x + 5.5
    tmp -= (x + 0.5) * np.log(tmp)
    series = 1.000000000190015
    for i, coefficient in enumerate([
        76.18009172947146, -86.50532032941677, 24.01409824083091,
        -1.231739572450155, 0.1208650973866179e-2, -0.5395239384953e-5,
    ]):
        series += coefficient / (x + i + 1)
    return -tmp + np.log(2.5066282746310005 * series / x)
```

Even the `for` loop is okay because it iterates a small, fixed number of times.

(It could be unrolled into 6 lines like

```python
        series += 76.18009172947146 / (x + 0 + 1)  # i = 0, coefficient = ...
```

instead of a loop.)

### The "iterate until converged" problem

How about another function (page 219): incomplete gamma $P(a, x)$?

$$P(a, x) = \frac{1}{\Gamma(a)} \int_0^x t^{a - 1} e^{-t} \, dt \hspace{0.5 cm} \mbox{where} \hspace{0.5 cm} a > 0$$

In [ ]:
SMALL_NUMBER = 3e-7

def incomplete_gamma_P(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta
        if np.absolute(delta) < np.absolute(summation) * SMALL_NUMBER:
            return summation * np.exp(-x + a*np.log(x) - gln)

    raise RuntimeError("did not converge")

<br>

In [ ]:
xs = np.linspace(0, 15, 10000)[1:]

In [ ]:
plt.plot(xs, [incomplete_gamma_P(0.5, x) for x in xs]);
plt.plot(xs, [incomplete_gamma_P(2.0, x) for x in xs]);
plt.plot(xs, [incomplete_gamma_P(10, x) for x in xs]);

In [ ]:
plt.plot(xs, scipy.special.gammainc(0.5, xs));
plt.plot(xs, scipy.special.gammainc(2.0, xs));
plt.plot(xs, scipy.special.gammainc(10, xs));

This one doesn't "just work" for arrays:

In [ ]:
incomplete_gamma_P(3.0, xs)

<br><br>

Python can't decide whether to enter the body of the `if` statement or not because the expression may be true for some elements of the array and false for others.

**What should we do?**

Have all values continue to iterate, even if some have already converged?

In [ ]:
def incomplete_gamma_P_numpy_keep_going(a, x):
    gln = log_of_gamma(a)

    delta = summation = 1.0 / a

    for i in range(100):
        delta *= x / (a + i + 1)
        summation += delta

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
plt.plot(xs, incomplete_gamma_P_numpy_keep_going(0.5, xs));
plt.plot(xs, incomplete_gamma_P_numpy_keep_going(2.0, xs));
plt.plot(xs, incomplete_gamma_P_numpy_keep_going(10, xs));

**What should we do?**

Keep track of which values have converged and don't compute those?

In [ ]:
SMALL_NUMBER = 3e-7

def incomplete_gamma_P_numpy_tricky_bookkeeping(a, x):
    gln = log_of_gamma(a)

    delta = np.full(x.shape, 1.0 / a)
    summation = delta.copy()

    not_converged = np.ones(x.shape, np.bool_)

    for i in range(100):
        delta[not_converged] *= x[not_converged] / (a + i + 1)
        summation[not_converged] += delta[not_converged]
        not_converged &= np.absolute(delta) >= np.absolute(summation) * SMALL_NUMBER

    return summation * np.exp(-x + a*np.log(x) - gln)

In [ ]:
plt.plot(xs, incomplete_gamma_P_numpy_tricky_bookkeeping(0.5, xs));
plt.plot(xs, incomplete_gamma_P_numpy_tricky_bookkeeping(2.0, xs));
plt.plot(xs, incomplete_gamma_P_numpy_tricky_bookkeeping(10, xs));

It's certainly simpler to let them all converge, although that means doing more computations than necessary.

So... which one's faster?

In [ ]:
%%timeit

incomplete_gamma_P_numpy_keep_going(3.0, xs)

<br>

In [ ]:
%%timeit

incomplete_gamma_P_numpy_tricky_bookkeeping(3.0, xs)

<br>

The _easy_ one is faster! This is not unusual, but not guaranteed. It's often better to let an algorithm churn unnecessarily than try to do careful convergence bookkeeping.

**Go to the [Part 2 project](project.ipynb) now!**